In [ ]:
!pip install -U tf-nightly==2.4.0.dev20200703
OUTPUT_DIR = 'output/'

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

from PHNetworks.PHOptimizer import PortHamiltonianOptimizer as PHOpt

from tf_plot_callback import Plotter
import matplotlib.pyplot as pyplot

In [ ]:
# Load MNIST handwritten digits dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Prescale pixel byte brightness to float in range [0, 1]
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert integer category to activation of the output neurons
y_train = keras.utils.to_categorical(y_train, 10)
y_test  = keras.utils.to_categorical(y_test, 10)

In [ ]:
def make_model():
    keras.backend.set_floatx('float64')
    model = keras.models.Sequential(
        layers=[
            # Make 28x28-entry two-dimensional input 784-entry one-dimensional
            layers.Flatten(input_shape=(28, 28), name='input_layer'),
            # Hidden layers: 250 nodes with Softplus activation function
            layers.Dense(250, activation='softplus', name='hidden_layer_1'),
            # Hidden layers: 250 nodes with Softplus activation function
            layers.Dense(250, activation='softplus', name='hidden_layer_2'),
            # Output layer: 10 nodes for 10 possible digits
            layers.Dense(10, activation='softmax', name='output_layer')
        ],
        name='mnist_model')
    return model

In [ ]:
def train_test(batch_size, ivp_duration):
    # Create Model
    model = make_model()
    model.compile(loss=keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

    # Create a new optimizer instance
    optimizer = PHOpt(ivp_period=ivp_duration, ivp_step_size=0.05, alpha=1.2, beta=0.25, gamma=1.5, resistive=0.65)

    # Initialize a plotter instance, jump through some hoops to
    # gain access to protected properties of the optimizer
    optimizer._check_model_and_state(model)
    harry = Plotter(optimizer.ivp_period)

    print(f'BEGINNING TRAINING WITH batch_size={batch_size} AND ivp_duration={ivp_duration}')

    # Fit the model
    optimizer.train(
        model, x_train, y_train,
        batch_size=batch_size,
        metrics=[keras.metrics.CategoricalAccuracy()],
        callbacks=[
            harry
        ],
        epochs=3,
        verbose=True
    )

    _, accuracy = model.evaluate(x_test, y_test, verbose=False)

    fig = harry.plot()

    fig.axes[1].set_title(f'Trainingsverlauf (batch_size = {batch_size}, $t^*$ = {ivp_duration})')

    name = f'batch-size-comparison_size={batch_size}_duration={ivp_duration}_accuracy={accuracy}.pdf'
    fig.savefig(OUTPUT_DIR + name)

    pyplot.close(fig)

    return accuracy


In [ ]:
BATCH_SIZES = [60000, 10000, 1000, 250, 50, 20, 1]
IVP_PERIODS = [0.1, 0.5, 2.0, 5.0, 20.0, 50.0]

for bs in BATCH_SIZES:
    for ivp_period in IVP_PERIODS:
      accuracy = train_test(bs, ivp_period)
      print(f'Reached accuracy of {accuracy} with batch_size={bs} and ivp_duration={ivp_period}')

In [ ]:
batch_size = 1
ivp_duration = 0.05

# Create Model
model = make_model()
model.compile(loss=keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

# Create a new optimizer instance
optimizer = PHOpt(ivp_period=ivp_duration, ivp_step_size=0.01, alpha=1.2, beta=0.25, gamma=1.5, resistive=0.65)

# Initialize a plotter instance, jump through some hoops to
# gain access to protected properties of the optimizer
optimizer._check_model_and_state(model)
harry = Plotter(optimizer.ivp_period)

print(f'BEGINNING TRAINING WITH batch_size={batch_size} AND ivp_duration={ivp_duration}')

# Fit the model
optimizer.train(
    model, x_train, y_train,
    batch_size=batch_size,
    metrics=[keras.metrics.CategoricalAccuracy()],
    callbacks=[
        harry
    ],
    epochs=1,
    verbose=True
)

_, accuracy = model.evaluate(x_test, y_test, verbose=False)

fig = harry.plot()

fig.axes[1].set_title(f'Trainingsverlauf (batch_size = {batch_size}, $t^*$ = {ivp_duration})')

name = f'batch-size-comparison_size={batch_size}_duration={ivp_duration}_accuracy={accuracy}.pdf'
fig.savefig(OUTPUT_DIR + name)

pyplot.close(fig)

print(f'Reached accuracy of {accuracy} with batch_size={batch_size} and ivp_duration={ivp_duration}')